In [1]:
#import librabries
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first='true')
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [6]:
dependent=dataset['Purchased']
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [7]:
# splitting training and test dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

C:\Users\sindhiya maria\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [9]:
from sklearn.svm import SVC
# Initialize SVC with probability=True
svc = SVC(probability=True)

from sklearn.model_selection import GridSearchCV

param_grid={'kernel':['rbf','poly','linear','sigmoid'],
            'C':[10,100,1000,2000,3000],'gamma':['scale', 'auto']}


grid = GridSearchCV(svc, param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(x_train, y_train) 

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['rbf', 'poly', 'linear', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [10]:
# print best parameter after tuning
re=grid.cv_results_
grid_predictions = grid.predict(x_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)

# print classification report 
from sklearn.metrics  import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}: 0.9100355779243318


In [12]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[80  5]
 [ 7 42]]


In [13]:
print("The Report:\n",clf_report)

The Report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [14]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9539015606242497

In [15]:
table =pd.DataFrame.from_dict(re)

In [16]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018660,0.009130,0.002616,0.005233,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
1,0.012928,0.002162,0.002502,0.005003,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
2,0.010135,0.008548,0.003889,0.005968,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
3,0.006438,0.001845,0.001102,0.002204,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.762677,0.738916,0.753180,0.778067,0.766556,0.759879,0.013172,31
4,0.009630,0.003383,0.002658,0.003261,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,3
5,0.004848,0.003792,0.003102,0.006203,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,11
6,0.019116,0.003375,0.000902,0.001803,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,21
7,0.008972,0.005734,0.000902,0.001803,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,39
8,0.011012,0.002919,0.001809,0.003348,100,scale,rbf,"{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,1
9,0.048418,0.014441,0.000000,0.000000,100,scale,poly,"{'C': 100, 'gamma': 'scale', 'kernel': 'poly'}",0.822092,0.787284,0.766556,0.924528,0.901744,0.840441,0.062370,13


In [17]:
age_input=float(input("Age:"))

Gender_male_input=int(input("Gender_Male 0 or 1:"))

EstimatedSalary_input=int(input("EstimatedSalary:"))


Age:19
Gender_Male 0 or 1:1
EstimatedSalary:19000


In [18]:
Future_Prediction=grid.predict([[age_input,Gender_male_input,EstimatedSalary_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
